In [32]:
from google_bigquery import bq_to_csv, csv_to_bq
import pandas as pd
import json
import yaml
import re
import glob
from datetime import date
import unidecode
import numpy as np
import string
from keyword import iskeyword

In [33]:
dataset_id = 'maxi_prices'
table_id ='price'
folder = str(date.today())
store = '/loblaws/'
path = '/Users/simonprudhomme/Desktop/maxi/'
path+folder+store

'/Users/simonprudhomme/Desktop/maxi/2020-04-01/loblaws/'

In [34]:
all_files = glob.glob(path+folder+store + "/*.csv")

li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

data = pd.concat(li, axis=0, sort=True,ignore_index=True)
print(data.shape)

(12047, 5)


In [35]:
def clean_attributs(attribut):
    s = attribut
    try:
        json_acceptable_string = s.replace("'", "\"")
        d = json.loads(json_acceptable_string)[0]
    except:
        d = yaml.load(s,Loader=yaml.FullLoader)[0]
    return d

data['attributs'] = [clean_attributs(attribut) for attribut in data['attributs']]

In [36]:
data_clean = pd.DataFrame()
for item in data['attributs']:
    sub_data = pd.DataFrame.from_dict(item,orient='index')
    data_clean = pd.concat([data_clean,sub_data],axis=1)
data_clean = data_clean.T.reset_index(drop=True)

data_clean['ratio'] = data['ratios']
data_clean['sales'] = data['sales']
data_clean['productCategory'] = data['name']
data_clean['date'] = data['date']

data_clean = data_clean.drop_duplicates().reset_index(drop=True)

In [37]:
def remove_ponctuation(x):
    if x is not np.nan:
        x = unidecode.unidecode(x)
        #x = x.translate(str.maketrans('', '', string.punctuation))
        x = x.replace(';\n;','')
        #x = x.strip()
    else:
        x = np.nan
    return x

data_clean['productName'] = [remove_ponctuation(x) for x in data_clean['productName']]

In [38]:
def is_numeric(x):
    if x is not np.nan:
        if any(i.isdigit() for i in x):
            return True
    else:
        return False

In [39]:
def clean_sale(x):
    if ' for ' in str(x):
        x = np.nan
    if ' MIN ' in str(x):
        x = np.nan
    if x is not np.nan :
        x = x.split('$')[1]
        x = x.strip()
        x = float(x)
    else:
        x = np.nan
    return x

data_clean['productSales'] = [clean_sale(x) if is_numeric(x) is True else np.nan for x in data_clean['sales']]

In [40]:
data_clean['productPrice'] = data_clean['productPrice'].astype(float)
data_clean['productPricePerGram'] = data_clean['ratio'].str.split('/')
data_clean['productUnit'] = data_clean['ratio'].str.split('/')

def clean_price(x):
    if x is not np.NaN:
        if len(x)>1 :
            try:
                x = x[0].replace('$','')
                x = x.replace(' ','')
                x = x.replace(',','.')
                x = float(x)
            except:
                x = np.nan
    else:
        x
    return x

def clean_unit(x):
    if x is not np.NaN:
        if len(x)>1 :
            x = x[1].strip()
    else:
        x
    return x

data_clean['productPricePerGram'] = [clean_price(x) for x in data_clean['productPricePerGram']]
data_clean['productUnit'] = [clean_unit(x) for x in data_clean['productUnit']]

In [41]:
data_clean['productURL'] = 'https://www.loblaws.ca/p/'+data_clean['productSKU']
data_clean['productSKU'] = data_clean['productSKU'].str.split('_')
data_clean['productSKU'] = [x[0] for x in data_clean['productSKU']]
data_clean['productSKU'] = data_clean['productSKU'].str.strip()
data_clean['productSKU']

0          20694522
1       20143381001
2       20174514001
3       20874331001
4       20840038001
           ...     
8254       21102499
8255       21004538
8256       20757538
8257       20799529
8258       21017324
Name: productSKU, Length: 8259, dtype: object

In [42]:
data_clean = data_clean[['productSKU', 'productName', 'productBrand', 'productPrice', 'dealBadge', 
            'productSales', 'productPricePerGram', 'productUnit','productURL','productCategory','date']]
data_clean['source'] = 'loblaws'

In [43]:
data_clean

,productSKU,productName,productBrand,productPrice,dealBadge,productSales,productPricePerGram,productUnit,productURL,productCategory,date,source
0,20694522,Spinach,New Star,4.99,None,NaN,1.10,100g,https://www.loblaws.ca/p/20694522_EA,vegetables,2020-04-01,loblaws
1,20143381001,Roma Tomatoes,None,0.87,None,NaN,7.25,1kg,https://www.loblaws.ca/p/20143381001_KG,vegetables,2020-04-01,loblaws
2,20174514001,Anjou Pears,None,1.74,None,NaN,7.25,1kg,https://www.loblaws.ca/p/20174514001_KG,vegetables,2020-04-01,loblaws
3,20874331001,Hot Finger Peppers,None,6.99,None,NaN,NaN,NaN,https://www.loblaws.ca/p/20874331001_EA,vegetables,2020-04-01,loblaws
4,20840038001,FM Grape Tomato 907g,Farmer's Market,5.99,None,NaN,0.66,100g,https://www.loblaws.ca/p/20840038001_EA,vegetables,2020-04-01,loblaws
...,...,...,...,...,...,...,...,...,...,...,...,...
8254,21102499,Loaf Soft Multigrain,UDI's,6.99,None,NaN,2.06,100g,https://www.loblaws.ca/p/21102499_EA,frozen,2020-04-01,loblaws
8255,21004538,Chicken Stir Fry Skillet Meals,Swanson,6.99,sale,1.0,0.59,100g,https://www.loblaws.ca/p/21004538_EA,frozen,2020-04-01,loblaws
8256,20757538,Loads Of Chocolatey Peanut Butter Treats Light...,President's Choice,6.99,None,NaN,0.47,100mL,https://www.loblaws.ca/p/20757538_EA,frozen,2020-04-01,loblaws
8257,20799529,Superfries Straight Cut,Mccain,3.99,None,NaN,0.61,100g,https://www.loblaws.ca/p/20799529_EA,frozen,2020-04-01,loblaws


In [44]:
filename_clean = str(date.today())+'_loblaws_clean.csv'
data_clean.to_csv(path+folder+'/'+filename_clean,index=False)